#### SageMaker Pipelines Lambda Step and Hugging Face

This notebook demonstrates how to use [SageMaker Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) to train and deploy a [Hugging Face](https://docs.aws.amazon.com/sagemaker/latest/dg/hugging-face.html) transformer using a Lambda function invoked by SageMaker Pipelines Lambda Step. 

Prerequisites:  
- Make sure your notebook environment has IAM managed policy `AmazonSageMakerPipelinesIntegrations` as well as `AmazonSageMakerFullAccess`
- For SageMaker Studio, use the kernel `Python 3 (Data Science)`

We'll start with some necessary imports.

In [ ]:
import boto3
import os
import numpy as np
import pandas as pd
import sagemaker
import sys
import time

from sagemaker.workflow.parameters import ParameterInteger, ParameterString
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)
from sagemaker.lambda_helper import Lambda

from sagemaker.sklearn.processing import SKLearnProcessor

from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import CacheConfig, ProcessingStep

from sagemaker.huggingface import HuggingFace, HuggingFaceModel

from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.step_collections import CreateModelStep, RegisterModel

from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)

from sagemaker.workflow.pipeline import Pipeline, PipelineExperimentConfig
from sagemaker.workflow.execution_variables import ExecutionVariables

Next, we'll perform some setup for SageMaker.

In [ ]:
region = sagemaker.Session().boto_region_name
sm_client = boto3.client("sagemaker")
boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sm_client
)

s3_prefix = "hugging-face-pipeline-demo"
base_job_prefix = s3_prefix
role = sagemaker.get_execution_role()

bucket = sagemaker_session.default_bucket()

#### Parameters

Before defining the pipeline, it is important to parameterize it.

In [ ]:
# processing step parameters
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.c5.2xlarge"
)
processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount", default_value=1
)

# training step parameters
training_instance_type = ParameterString(
    name="TrainingInstanceType", default_value="ml.p3.2xlarge"
)
training_instance_count = ParameterInteger(
    name="TrainingInstanceCount", default_value=1
)

# endpoint parameters
endpoint_instance_type = ParameterString(
    name="EndpointInstanceType", default_value="ml.g4dn.xlarge"
)

output_destination = "s3://{}/{}/data".format(bucket, s3_prefix)

cache_config = CacheConfig(enable_caching=False, expire_after="30d")

model_package_group_name = "HuggingFaceModelPackageGroup"

#### Data Preparation

The SKLearn Processing is used to invoke a SageMaker Processing job with a custom python script - `preprocessing.py`. 

In [ ]:
sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1",
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name=base_job_prefix + "/preprocessing",
    sagemaker_session=sagemaker_session,
    role=role,
)

step_process = ProcessingStep(
    name="ProcessDataForTraining",
    cache_config=cache_config,
    processor=sklearn_processor,
    outputs=[
        ProcessingOutput(
            output_name="train",
            destination="{}/train".format(output_destination),
            source="/opt/ml/processing/train",
        ),
        ProcessingOutput(
            output_name="test",
            destination="{}/test".format(output_destination),
            source="/opt/ml/processing/test",
        ),
        ProcessingOutput(
            output_name="validation",
            destination="{}/test".format(output_destination),
            source="/opt/ml/processing/validation",
        ),
    ],
    code="./scripts/preprocessing.py",
)

#### Create the Hugging Face Estimator

Use SageMaker's [Hugging Face](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html) Estimator class to create train the Hugging Face [BERT](https://huggingface.co/distilbert-base-uncased) model.

In the cell below, replace the container URI with the one for your region, as found in the table https://github.com/aws/deep-learning-containers/blob/master/available_images.md.  

In [ ]:
'''
container = sagemaker.image_uris.retrieve(
                                        framework="huggingface",
                                        region=boto3.Session().region_name,
                                        version="4.6.1",
                                        py_version="py36",
                                        base_framework_version="pytorch1.7.1",
                                        instance_type="ml.p3.2xlarge",
                                        image_scope="training" )
'''

container = '763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:1.7.1-transformers4.6.1-gpu-py36-cu110-ubuntu18.04' 

In [ ]:
hyperparameters = {
    "epochs": 1,
    "train_batch_size": 32,
    "model_name": "distilbert-base-uncased",
}

estimator = HuggingFace(
    image_uri=container,
    entry_point="train.py",
    source_dir="./scripts",
    base_job_name=base_job_prefix + "/training",
    instance_type=training_instance_type,
    instance_count=training_instance_count,
    role=role,
    transformers_version="4.6.1",
    pytorch_version="1.7.1",
    py_version="py36",
    hyperparameters=hyperparameters,
    sagemaker_session=sagemaker_session,
)

In [ ]:
step_train = TrainingStep(
    name="TrainHuggingFaceModel",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri
        ),
        "test": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri
        ),
    },
    cache_config=cache_config,
)

#### Model Evaluation

A ProcessingStep is used to evaluate the performance of the trained model. Based on the results of the evaluation, the model is created and deployed.

In the training job, the model was evaluated against the test dataset, the result of the evaluation was stored in the `model.tar.gz` file saved by the training job, the results of that evaluation are copied into a `PropertyFile` in this ProcessingStep so that it can be used in the ConditionStep. 

In [ ]:
script_eval = ScriptProcessor(
    image_uri=container,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name=base_job_prefix + "/evaluation",
    sagemaker_session=sagemaker_session,
    role=role,
)

evaluation_report = PropertyFile(
    name="AbaloneEvaluationReport",
    output_name="evaluation",
    path="evaluation.json",
)

step_eval = ProcessingStep(
    name="HuggingfaceEvalLoss",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="evaluation",
            source="/opt/ml/processing/evaluation",
            destination=f"s3://{bucket}/{s3_prefix}/evaluation_report",
        ),
    ],
    code="./scripts/evaluate.py",
    property_files=[evaluation_report],
    cache_config=cache_config,
)

#### Register the model

The trained model is registered into the Model Registry under a Model Package Group. The model is registered in the "Approved" state so that it can be deployed by the Lambda function. Registration will only happen if the output of the ConditionStep is true, i.e, the metrics being checked are within the threshold defined.

In [ ]:
model = HuggingFaceModel(
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    role=role,
    transformers_version="4.6.1",
    pytorch_version="1.7.1",
    py_version="py36",
    sagemaker_session=sagemaker_session,
)

step_register = RegisterModel(
    name="HuggingFaceRegisterModel",
    model=model,
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=["ml.g4dn.xlarge", "ml.m4.xlarge"],
    transform_instances=["ml.g4dn.xlarge", "ml.m4.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status="Approved",
)

#### Lambda Step

The SageMaker SDK provides a Lambda helper class that can be used to create a Lambda function. This function is provided to the LambdaStep for invocation via the Pipeline. Alternatively, a pre-defined Lambda function can be provided to the LambdaStep. 

The SageMaker Execution Role requires the policy `AmazonSageMakerPipelinesIntegrations` to create the Lambda function and the Lambda function needs a role with policies for creating a SageMaker endpoint. 

A helper function in `iam_helper.py` is provided to create the Lambda role, the use the script, the notebook execution role must the required policy to create a role. 

In [ ]:
%%writefile lambda_deployer.py

"""
This Lambda function creates an Endpoint Configuration and deploys a model to an Endpoint. 
The name of the model to deploy is provided via the `event` argument
"""

import json
import boto3


def lambda_handler(event, context):
    """ """
    sm_client = boto3.client("sagemaker")

    # The name of the model created in the Pipeline CreateModelStep
    model_name = event["model_name"]
    model_package_arn = event["model_package_arn"]
    endpoint_config_name = event["endpoint_config_name"]
    endpoint_name = event["endpoint_name"]
    role = event["role"]
    bucket = event["bucket"]
    s3_prefix = event["s3_prefix"]

    container = {"ModelPackageName": model_package_arn}

    create_model_respose = sm_client.create_model(
        ModelName=model_name, ExecutionRoleArn=role, Containers=[container]
    )

    create_endpoint_config_response = sm_client.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[
            {
                "VariantName": "variant1",
                "ModelName": model_name,
                "InstanceType": "ml.m5.xlarge",
                "InitialInstanceCount": 1
            }
        ],
        AsyncInferenceConfig={
            "OutputConfig": {
                "S3OutputPath": f"s3://{s3_bucket}/{s3_prefix}/output",
            },
            "ClientConfig": {
                "MaxConcurrentInvocationsPerInstance": 4
            }
        }
    )

    create_endpoint_response = sm_client.create_endpoint(
        EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
    )

    return {
        "statusCode": 200,
        "body": json.dumps("Created Endpoint!"),
        "other_key": "example_value",
    }

In [ ]:
# Create an IAM role using the `iam_helper` script or provide an existing IAM role to be used by the Lambda function

from iam_helper import create_lambda_role

lambda_role = create_lambda_role("lambda-deployment-role") 

In [ ]:
# Define the Lambda Step

# Use the current time to define unique names for the resources created
current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())

model_name = "demo-hf-model" + current_time
endpoint_config_name = "demo-hf-endpoint-config" + current_time
endpoint_name = "demo-hf-endpoint-" + current_time

function_name = "sagemaker-demo-hf-lambda-step" + current_time

# Lambda helper class can be used to create the Lambda function
func = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="lambda_deployer.py",
    handler="lambda_deployer.lambda_handler",
    timeout=600,
    memory_size=10240,
)

# The dictionary retured by the Lambda function is captured by LambdaOutput, each key in the dictionary corresponds to a 
# LambdaOutput

output_param_1 = LambdaOutput(
    output_name="statusCode", output_type=LambdaOutputTypeEnum.String
)
output_param_2 = LambdaOutput(
    output_name="body", output_type=LambdaOutputTypeEnum.String
)
output_param_3 = LambdaOutput(
    output_name="other_key", output_type=LambdaOutputTypeEnum.String
)

# The inputs provided to the Lambda function can be retrieved via the `event` object within the `lambda_handler` function 
# in the Lambda
step_deploy_lambda = LambdaStep(
    name="LambdaStepHuggingFaceDeploy",
    lambda_func=func,
    inputs={
        "model_name": model_name,
        "endpoint_config_name": endpoint_config_name,
        "endpoint_name": endpoint_name,
        "model_package_arn": step_register.steps[0].properties.ModelPackageArn,
        "role": role,
        "bucket": bucket,
        "s3_prefix": s3_prefix
    },
    outputs=[output_param_1, output_param_2, output_param_3],
)

In [ ]:
# The evaluated loss of the hugging face model must be less than 0.3 for the condition to be True and the subsequent steps
# to run

cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step=step_eval,
        property_file=evaluation_report,
        json_path="eval_loss",
    ),
    right=0.3,
)

step_cond = ConditionStep(
    name="CheckHuggingfaceEvalLoss",
    conditions=[cond_lte],
    if_steps=[step_register, step_deploy_lambda],
    else_steps=[],
)

In [ ]:
pipeline = Pipeline(
    name=f"HuggingFaceDemoPipeline" + "-" + current_time,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        training_instance_type,
        training_instance_count,
    ],
    steps=[step_process, step_train, step_eval, step_cond],
    sagemaker_session=sagemaker_session,
)

In [ ]:
import json

json.loads(pipeline.definition())

In [ ]:
pipeline.upsert(role_arn=role)

In [ ]:
execution = pipeline.start()

In [ ]:
execution.wait()

#### Prediction

After the previous cell completes, you can check whether the endpoint has finished deploying.  In SageMaker Studio, click on the tilted triange icon in the left toolbar, then select **Endpoints** in the drop down menu.  In the list of endpoints, click on the one with the name beginning `demo-hf-endpoint`, then click on **AWS settings**.  When the Status becomes InService, you can run the following code cells.

In [ ]:
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer

hf_predictor = sagemaker.predictor.Predictor(endpoint_name, 
                                             sagemaker_session=sagemaker_session,
                                             serializer=JSONSerializer(),
                                             deserializer=JSONDeserializer() )

In [ ]:
sentiment_input1 = {"inputs":"Although the movie had some plot weaknesses, it was engaging. Special effects were mind boggling."}

hf_predictor.predict(sentiment_input1)

In [ ]:
sentiment_input2 = {"inputs":"There was some good acting, but the story was ridiculous. The other sequels in this franchise were better."}

hf_predictor.predict(sentiment_input2)

#### Cleanup Resources

The following cell will delete the resources created by the Lambda function and the Lambda itself. 
Deleting other resources such as the S3 bucket and the IAM role for the Lambda function are the responsibility of the notebook user. 

In [ ]:
# Delete the Lambda function
func.delete()

# Delete the endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)

# Delete the EndpointConfig
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

# Delete the model
sm_client.delete_model(ModelName=model_name)